# COS360 - Otimização 2020.2
## Bernardo Maiorano e Igor Amaral

$$
\begin{align}
\displaystyle \min \ \  &f(x)\\
\text{sujeito a:}\  \ \ &x \in \Omega\\
\\
f(x_1,x_2,x_3,x_4) &= -30x_1 - 10x_2 - 40x_3 - 12x_4\\
\Omega = \{x \in \mathbb{R}^4 :& \  {33x_1 + 14x_2 + 47x_3 + 11x_4 \le 59\}} \\
\end{align}
$$

1) fazer um estudo da função

In [1]:
import numpy as np

In [2]:
coefficients = np.array([-30, -10, -40, -12])
omega = np.array([33, 14, 47, 11])

def f(x):
    return x @ coefficients

def G(x):
    # Restrição de desigualdade
    return omega @ x <= 59

# Gradientes constantes
grad_f = coefficients

In [3]:
# mínimo pelo simplex: x = [0,0,0,59/11]
f([0,0,0,59/11])

-64.36363636363636

Penalidade Exterior

Para transformar nosso problema restrito em irrestrito. Temos que usar o método de penalidade externa ou interna.
$ \\ $
Assim, o problema transformado seria: $\min \phi(x,\rho) = f(x) + \rho P(x)$ onde $\rho$ é o parâmetro de penalidade e $P(x)$ é a função de penalidade.

Vamos usar a função de penalidade externa: $\displaystyle P(x) = \sum_{h \in H} h(x)^2 + \sum_{g \in G} \max[0, g(x)]^2$

No nosso caso, a função objetivo torna-se: $ \\ $
$$
{-30x_1-10x_2-40x_3-12x_4} + \rho \max[0,{33x_1+14x_2+47x_3+11x_4-59}]^2
$$

In [4]:
def P_Exterior(x, p = 1000):
    return f(x) + p*pow(max(0, omega@x-59),2)

# https://www.wolframalpha.com/input/?i=derivate+%28p*%28max+%280%2C+%2833x%2B14y%2B47z%2B11w-59%29%29%29%5E2
# grad_Pext = grad_f + p*grad(Penalidade)
def grad_Pext(x, p= 1000):
    if G(x): return grad_f
    return np.array([(grad_f[i] + (p*2*omega[i]*(x@omega-59))) for i in range(4)])

# https://www.wolframalpha.com/input/?i=hessian+%28%28-30x-10y-40z-12w%29+%2B+%28p*%2833x%2B14y%2B47z%2B11w-59%29%5E2%29%29+with+respect+to+%28x%2Cy%2Cz%2Cw%29
def hessian_Pext(x, p= 1000):
    hessian = np.zeros((4,4))
    if G(x): return hessian
    for i in range(4):
        hessian[i] = [(p*2*omega[i]*omega[j]) for j in range(4)]
    return hessian

In [5]:
z = np.array([0,0,0,0])
P_Exterior(z), grad_Pext(z)#, hessian_Pext(k,1), np.linalg.inv(hessian_Pext(k,1))

(0, array([-30, -10, -40, -12]))

Penalidade Interior

In [6]:
# Função de penalidade interior: f(x) - p/G(x)
def P_Interior(x, p = 0.0001):
    # p: parâmetro de penalidade
    return f(x) - (p/(omega@x-59))

# https://www.wolframalpha.com/input/?i=derivate+%28%28-30x-10y-40z-12w%29-%28p%2F%2833x%2B14y%2B47z%2B11w-59%29%29%29
# grad_Pin = grad_f - p*grad(Penalidade)
def grad_Pin(x, p = 0.0001):
    return np.array([(grad_f[i] - p*(-omega[i]/pow(omega@x-59, 2))) for i in range(4)])

In [7]:
P_Interior(z), grad_Pin(z)

(1.6949152542372882e-06,
 array([-29.99999905,  -9.9999996 , -39.99999865, -11.99999968]))

In [8]:
def armijo(x, direction, f, Grad):
    '''
    '''
    t = 1
    gamma = 0.8
    n = 0.25
    n_iter = 0
    
    while (f(x + t*direction) > f(x) + n*t*Grad(x)@direction):
        t = t*gamma
        n_iter += 1
        
#     print("Armijo:",t,n_iter)
    return t, n_iter

In [9]:
def stop(x, prev_x, n_iter, max_iters = 50):
    return np.allclose(x, prev_x, atol=1e-16) or n_iter > max_iters
 
def Gradiente(x, f, Grad):
    
    #ponto inicial fora do viável
    if not G(x):
        print('fora do viável')
        return
    
    k = 0
    n_iter = 0
    prev_x = float('inf')*np.ones(4)
    while not stop(x,prev_x,n_iter):
        direction = -Grad(x)
        t,_ = armijo(x,direction, f, Grad)
        prev_x = x
        x = x + t*direction
        k += 1
        n_iter += 1
#         print(f(x),x)
    return x, f(x), n_iter

In [10]:
x = np.array([0,0,0,0], dtype=np.float64)
Gradiente(x, P_Exterior, grad_Pext)
# Gradiente(x, P_Interior, grad_Pin)

(array([0.56334641, 0.18776653, 0.75111295, 0.22535001]),
 -51.52667222900208,
 8)

In [11]:
# def Newton(x, f, Grad, Hessian):
#     k = 0
#     n_iter = 0
#     prev_x = float('inf')*np.ones(4)
#     while not stop(x,prev_x,n_iter):
#         direction = -(inverse(Hessian)@Grad(x))

# def quase_Newton(x,f,Grad,Hessian):
#     k = 0
#     n_iter = 0
#     prev_x = float('inf')*np.ones(4)
#     while not stop(x,prev_x,n_iter):
#         direction = 